# Final Function using Universal Vector Encoder

In [7]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
import pickle
import sys

# loading the text dictionary file
with open('text_dictionary.pickle', 'rb') as temp_var:
    total_text_dictionary = pickle.load(temp_var)

# code reference from the elastic search documentation 
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
if es.ping():
    print('Connected to ES!')
else:
    print('Could not connect!')
    sys.exit()

# loading the encoder model
import tensorflow as tf
embed = tf.keras.models.load_model('universal-sentence-encoder_4')
def make_vector(query):
    embeddings = embed([query])
    vector = []
    for i in embeddings[0]:
        vector.append(float(i))
    return vector

# connecting to elastic search

# definning a function to normalize the score values of the result.


def final_func1(query):
    def norm_list(lis):
        scores = [x[0] for x in lis]
        ma = max(scores)
        mi = min(scores)
        for i in range(len(lis)):
            lis[i][0] = (lis[i][0] - mi)/(ma - mi)
        return lis
    
    
    request={
            'query':{ 'match':{"total_texts":query } }
            }

    res= es.search(index='database',body=request)
    l1 = []
    for hit in res['hits']['hits']:
        l1.append([hit['_score'] , hit['_id']])
# change the cosine similarity to euclidean distance

    query_vector = make_vector(query)
    request = {"query" : {
                "script_score" : {
                    "query" : {
                        "match_all": {}
                    },
                    "script" : {
                        #"source": "cosineSimilarity(params.query_vector, 'total_vectors') + 1.0",
                        "source": "1 / (1 + l2norm(params.query_vector , 'total_vectors'))" ,
                        "params": {"query_vector": query_vector}
                    }
                }
             }
    }

    res= es.search(index='database',body=request)
    l2 = []
    for hit in res['hits']['hits']:
        l2.append([hit['_score'] , hit['_id']])
    
    l1 = norm_list(l1)
    l2 = norm_list(l2)
    
    # getting the weighted average score for the text search and semantics search
    temp_doc = {}
    for i in l1:
        temp_doc[i[1]]  = i[0]*2
    for i in l2:
        temp_doc[i[1]] = temp_doc.get(i[1] , 0) + i[0]*5
    
    inverse_temp_doc = [(i[1] , i[0])  for i in temp_doc.items()]
    inverse_temp_doc = sorted(inverse_temp_doc , reverse = True)
    return inverse_temp_doc[:10]


# Printing the Results

In [11]:
# getting the combined search results both semantic and the text based.
answer_no = 1
for i in final_func1(" what is a linked list ? "):
    print('-'*50 + "Answer No:" + str(answer_no) + '-'*50)
    title = total_text_dictionary[i[1]][0]
    question = total_text_dictionary[i[1]][1]
    print("title : " ,title )
    print('\n')
    print("question :" , question)
    print('\n')
    sub_answer = 1
    for i in total_text_dictionary[i[1]][2:]:
        print("subanswer " + str(sub_answer) +' : ' + i )
        sub_answer+=1
    answer_no+=1

--------------------------------------------------Answer No:1--------------------------------------------------
title :  finding an element in a infinite unsorted doubly linked list


question : is there any way to find an element in an unsorted doubly linked list given an element and pointer from which we can navigate? (we can't use head/tail pointers since the list is infinite)



subanswer 1 : infinite/unbounded in both directions?
zig-zag.

--------------------------------------------------Answer No:2--------------------------------------------------
title :  a question about linked list


question : i would like to check the answer of this exercise:
a circular doubled linked list with n elements has pointers that cost k bytes each of space in memory. how many bytes do the pointers of this list cost in total?
researching about circular linked lists, i can assume that each node has 2 pointers. so am i right to say that the answer is 2(n)k bytes?
thanks.



subanswer 1 : the list tak

## Actual Results from stack overflow for the above question

<img src = "stack_image.png">